# JETANK Servos

JETANK는 제트봇을 기반으로 한 오픈 소스 로봇 제품입니다. 제트봇을 기반으로 5개의 서보와 한 쌍의 크롤러를 추가하여 투구할 수 있는 카메라와 물체를 잡을 수 있는 로봇 팔을 제공합니다.

로봇 암을 제어하기 전에 서보가 드라이버 보드와 연결되어 있고 튜토리얼 내용에 따라 각 서보의 초기 위치가 올바르게 보정되었는지 확인합니다. 서보가 올바르게 연결되어 전원이 켜지면 서보를 돌릴 때 분명한 감쇠가 발생합니다.

다음 문서에서는 파이썬을 사용하여 서보를 제어할 것입니다.


### TTLservo 클래스 가져오기

먼저 servos를 제어하는 데 사용되는 라이브러리를 가져와야 합니다. 이 코드를 실행하기 전에 ttYTHS1(Jetson Nano와 servos 간의 통신을 위한 직렬 포트)은 기본적으로 Jetson Nano 시스템 자체에서 점유하고 있습니다. 설치 스크립트가 정상적으로 모든 것을 실행하면 이전에 ttYTHS1을 점유했던 서비스가 종료되고 스크립트는 자동으로 ttYTHS1에 0666 권한을 추가합니다.



이전 설치 스크립트를 실행하지 못하면 여기에서 tTYTHS1을 사용할 때 사용 권한 거부 오류가 발생할 수 있습니다. 이 오류가 발생하면 다음과 같은 해결 방법이 있습니다:
- JupyterLab의 왼쪽 상단에 있는 플러스 기호를 클릭하면 새 Launcher 탭이 열리고 이 탭의 Terminal이 Jetson Nano 콘솔이 열립니다.
- 콘솔에 다음을 입력하고 Enter 키를 누릅니다.
    > sudo chmod 666 /dev/ttyTHS1
- Jetpack의 기본 비밀번호 'jetbot'을 입력하고 Enter 키를 눌러 수정 권한을 확인합니다.
- 왼쪽의 'JEANK_1_servos.ipynb'를 마우스 오른쪽 버튼으로 클릭하고 'Shut Down Kernal'을 클릭합니다.
- 현재 'JEANK_1_servos.ipynb' 탭을 닫고 왼쪽 'JEANK_1_servos.ipynb'를 다시 두 번 클릭하여 새 Kernal을 실행합니다.

그런 다음 다음 다음 코드를 선택하고 Ctrl+Enter를 눌러 servos를 제어하는 데 사용되는 라이브러리를 가져올 수 있습니다. 다음 프롬프트가 나타납니다:
포트를 여는 데 성공했습니다
보레이트 변경 성공

In [1]:
from SCSCtrl import TTLServo

Succeeded to open the port
Succeeded to change the baudrate


다음으로 가장 간단한 서보 제어 기능인 'servoAngleCtrl()'을 사용하여 서보를 제어합니다. 이 기능을 사용하여 특정 서보의 회전을 제어할 수 있습니다. 이 기능을 호출할 때는 다음과 같은 네 가지 매개 변수가 필요합니다:
'servoAngleCtrl(servoID, angleInput, 방향, 속도)'

> **servoID:** 제어되는 서보의 번호. 로봇팔(카메라)의 수평방향(Pan) 회전에 대한 서보 번호는 1번, 로봇팔 뿌리의 피칭 이동에 대한 서보 번호는 2번, 로봇팔 중간관절의 피칭 이동에 대한 서보 번호는 3번, 4번 서보는 물체를 잡는 데 사용되며, 5번 서보는 카메라의 틸트(Tilt) 이동에 사용됩니다.

> **angleInput:** 서보의 회전 각도를 제어합니다. 이론적으로 범위는 +-150이 될 수 있습니다. 사실 기계 구조의 간섭을 고려하면 회전 범위는 제한될 것입니다. 여기서 angleInput의 각도는 스티어링 기어의 중심(초기) 위치에 상대적입니다. 스티어링 기어를 중심(초기) 위치로 회전시키기를 원한다면 angleInput의 값을 0으로 두십시오.

> **direction:** 이 파라미터의 값은 1 또는 -1이 될 수 있습니다. 이 파라미터는 'TTLservo.servOangleCtrl(4, -40, 1, 150)', 'TTLservo.servOangleCtrl(4, 40, -1, 150)' 등 서보의 회전 방향을 조절하는 데 사용됩니다.

> **speed:** 이 파라미터는 서보의 회전속도를 조절하는데 사용됩니다. 이 값이 클수록 회전속도가 빠릅니다. 범위는 1-1500이지만 이 파라미터가 0일 때 최대 회전속도도 나타낸다는 점에 유의해야 합니다.

아래 코드 셀을 실행하여 모든 서보가 150의 속도로 초기 위치로 회전하도록 합니다.

In [2]:
TTLServo.servoAngleCtrl(1, 0, 1, 150)
TTLServo.servoAngleCtrl(2, 0, 1, 150)
TTLServo.servoAngleCtrl(3, 0, 1, 150)
TTLServo.servoAngleCtrl(4, 0, 1, 150)
TTLServo.servoAngleCtrl(5, 0, 1, 150)

512

### 해결된 모션 컨트롤

두 서보의 결합으로 로봇 팔의 척 위치가 제어되기 때문에 두 서보의 회전을 따로 제어하면 로봇 팔의 척 위치를 제어하기가 어려워집니다. 따라서 역운동학 함수를 사용하여 척의 위치를 계산합니다. 결합된 두 서보의 회전 각도는 로봇 팔의 움직임을 제어하고자 하는 척이 있는 위치의 상대 좌표 값 '(x, y)'만 입력하면 됩니다.

다음 코드를 실행하여 '(100,0)'의 위치로 기계식 암척의 이동을 제어하고, 100은 X 값, X 값이 클수록 척 위치를 전진시키고, 0은 Y 값, Y 값이 클수록 위치가 증가하며, 단위는 밀리미터입니다.

In [3]:
TTLServo.xyInput(100, 0)

[-114.6243183521641, 55.377838160268176]

X값과 Y값의 관계를 보다 직관적으로 보여주기 위해 로봇팔을 '(100, 0)'에서 '(200, 0)'으로 단계적으로 이동시킨 후 다시 '(150, 0)' 위치로 이동시킵니다.
여기서 이동하면 위치의 X 파라미터가 변경됩니다.
로봇 암의 움직임을 관찰하면서 다음 코드를 실행합니다.

In [4]:
import time

# Move to position（100，0）
TTLServo.xyInput(100, 0)
time.sleep(1)

# from（100，0）to（200，0）step by step
for i in range(100, 201):
    TTLServo.xyInput(i, 0)
    time.sleep(0.01)
time.sleep(1)

# Smooth back and forth movement between（100，0）and（100，100）,The third parameter is the time required to move to the position, in seconds.
for i in range(0,4):
    TTLServo.xyInputSmooth(100, 0, 1)
    time.sleep(2)
    TTLServo.xyInputSmooth(100, 100, 1)
    time.sleep(2)

# Move to position（150，0）
TTLServo.xyInput(150, 0)
time.sleep(1)

다음 코드는 Y 값을 변경하려면 다음 코드를 실행할 때 다음 코드를 실행할 때 다음 코드를 실행합니다.

In [5]:
# Move to position（150，-50）
TTLServo.xyInput(150, -50)
time.sleep(1)

# from（150，-50）to（150，50）step by step
for i in range(-50, 51):
    TTLServo.xyInput(150, i)
    time.sleep(0.01)
time.sleep(1)

# Move to position（150， 0）
TTLServo.xyInput(150, 0)
time.sleep(1)

서보를 사용할 때는 회전만 시작하고 회전 각도는 결정되지 않을 때가 있습니다. 예를 들어 servoAngleCtrl(1, 80, 1, 150)을 사용하여 카메라가 오른쪽으로 회전하도록 제어한 다음 servoStop(1)을 호출하여 서보 1번을 멈출 수 있습니다. 위의 80은 제한 각도를 설정하는 데 사용할 수도 있습니다.

In [6]:
TTLServo.servoAngleCtrl(1, 80, 1, 150)
time.sleep(2)
TTLServo.servoStop(1)

위는 스티어링 기어의 주요 용도입니다.